In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_astradb import AstraDBVectorStore
from langchain.agents import AgentType,load_tools,initialize_agent
from dotenv import load_dotenv
import os

c:\a\envs\medical\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(temperature = 0.4,model = "gemini-pro")

In [3]:
print(llm.invoke("Who are you?.").content)

I am Gemini, a multi-modal AI language model developed by Google. I am designed to provide information and assist with a wide range of language-related tasks, such as answering questions, generating text, translating languages, and writing different kinds of creative content.


In [4]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob = "*.pdf",loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_documents = load_pdf("data/")
len(extracted_documents)

637

In [7]:
def split_chunks(extracted_documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap = 20)
    text_chunks = splitter.split_documents(extracted_documents)
    return text_chunks

In [8]:
text_chunks = split_chunks(extracted_documents)
print(len(text_chunks),"\n")
print(text_chunks[:5])

7020 

[Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 1}, page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'), Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 2}, page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1'), Document(metadata={'source': 'data\\Medical_book.pdf', 'page': 3}, page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and'), Document(metadata={'source': 'data\\Medical_boo

In [9]:
ASTRA_DB_API = os.getenv("ASTRA_TOKEN")
ASTRA_ENDPOINT = os.getenv("DB_ENDPOINT")
def ingest(status):
    vector_store = AstraDBVectorStore(token = ASTRA_DB_API,
                                      api_endpoint = ASTRA_ENDPOINT,
                                      embedding = gemini_embeddings,
                                      namespace = "default_keyspace",
                                      collection_name = "Medical")
    is_full = status
    if is_full == None:#THIS MEANS THERE IS NO VECTORS CREATED IN DB
        inserted_ids = vector_store.add_documents(text_chunks)
    else:
        return vector_store
    
    
    return vector_store,inserted_ids

    

In [10]:
vector_store = ingest("done")#None cuz we are making the DB for first time
# print(f"inserted {len(inserted_ids)} documents.")

In [15]:
results = vector_store.similarity_search("What is pneumonia?")
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* heal, but rather becomescarred, a process known as fibrosis. The lack of a nor-mal amount of oxygen causes the blood vessels of thelung to become narrower, and in time they, too, maybecome scarred and filled with clotted blood. The lungsas a whole become very “stiff,” and it becomes muchharder for the patient to breathe. [{'source': 'data\\Medical_book.pdf', 'page': 80}]
* Coughing brings up a greenish yellow phlegm or sputum.These symptoms may be accompanied by a fever of up
to 102°F (38.8°C). Wheezing after coughing is common.
In uncomplicated acute bronchitis, the fever and [{'source': 'data\\Medical_book.pdf', 'page': 611}]
* inflammations of the air passages, their causes and treat-ments are different. Acute bronchitis is most prevalent inwinter. It usually follows a viral infection, such as a coldor the flu, and can be accompanied by a secondary bacter-ial infection. Acute bronchitis resolves within two weeks,although the cough may persist longer. Acute bronchitis,
like any upp

In [16]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

template = """You are a medical Chatbot who is expert in medicine knowledge and cure Context: {context}
Question: {question} 
"""

# template="""
# Use the following pieces of information to answer the user's question.

# Context: {context}
# Question: {question}

#  .If user asks for medication then give name of them
# Helpful answer:
# """

prompt=ChatPromptTemplate.from_template(template)


In [17]:
chain = ({"context":retriever,"question":RunnablePassthrough()}|prompt|llm|StrOutputParser())
response = chain.invoke("What is Malaria?Prescribe some medicines")

In [18]:
print(response)

In [15]:
serp_api_key = os.getenv("SERPAPI_API_KEY")
if not serp_api_key:
    raise ValueError("SERP API key not found. Ensure 'SERP_KEY' is set in the environment variables.")

tool = load_tools(["serpapi"],serp_api_key = serp_api_key,llm = llm)
agent = initialize_agent(tools = tool,llm = llm,agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [17]:
agent.invoke("Best hospital in visakhapatnam")



> Entering new AgentExecutor chain...
Action: Search
Action Input: Best hospital in visakhapatnam
Observation: ['1429 Hospitals in Visakhapatnam. Find ✓Multispeciality Hospitals, ✓Private Hospitals, ✓Dental Hospital , ✓Government Hospitals in Visakhapatnam.', 'Best Hospitals in Visakhapatnam. Apollo Healthcity Hospital. Ram Nagar, Visakhapatnam. The fastest-growing liver transplant program ...', 'Hospitals in Visakhapatnam · Padmasri Hospital & IVF Centre · Apollo Health City · Apollo Healthcity Hospital · Asian Institute of Nephrology and Urology.', 'CARE Hospitals is one of the best hospitals in Visakhapatnam/Vizag. We provide services in over 30 specialities like Cardiology, Cardiothoracic Surgery, ...', "Queen's NRI Hospital ranks among the best hospitals in Vishakhapatnam. With experience of over 25 years in service, we are committed to ...", 'Private hospitals like St. Stephens Hospital, Vadamalayan Hospitals, and Apollo Specialty Hospital are known for their state-of-the-art f

{'input': 'Best hospital in visakhapatnam',
 'output': 'Some of the best hospitals in Visakhapatnam are Apollo Hospitals Ramnagar Vizag, CARE Hospitals, Visakhapatnam, and Medicover Hospitals.'}